<a href="https://colab.research.google.com/github/Romqa41/GPN/blob/main/%D0%A3%D0%BF%D1%80%D0%BE%D1%89%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9_%D1%80%D0%B5%D0%B5%D1%81%D1%82%D1%80_FCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок импорта библиотек

In [ ]:
import pandas as pd
import numpy as np
import math
import re
from datetime import datetime

# Блок функций

Функция замены латинских букв на кириллические

In [ ]:
def replace_letters(text):
  # создаем словарь соответствия латинских букв и кириллицы
  mapping = {
      'A': 'А',  # пример: латинская A -> кириллическая А
      'E': 'Е',
      'K': 'К',
      'M': 'М',
      'O': 'О',
      'T': 'Т',
      'C': 'С',
      'P': 'Р',
      'X': 'Х'
      }
  # заменяем каждую букву из текста, если она есть в словаре
  return ''.join(mapping.get(char, char) for char in text)

Функция для поиска автомобильных номеров

In [ ]:
def transport_number(text):
  # проверка на наличие данных
  if pd.notna(text):
    text = str(text)

    # корректировка номеров ТС - удаление пробелов, (, ) и /, приведение к верхнему регистру
    processed_text = re.sub(r'[\s\(\)\/\-\\]', '', str(text)).upper()

    processed_text = replace_letters(processed_text)

    # паттерн для автомобильного номера
    pattern = r'[А-ЯA-Z]\d{3}[А-ЯA-Z]{2}\d{0,3}'

    # ищем совпадение по паттерну, если находим - функция возвращает весь номер
    res = re.search(pattern, processed_text)
    return res.group() if res else None

  # в случае отсутствия искомого паттерна функция ничего не возвращает
  return None

Функция для поиска транспортных накладных

In [ ]:
def transport_invoice(text):
  # проверка на наличие данных
  if pd.notna(text):
    text = str(text)

    # удаляем id
    text = re.sub(r'\{\d{10}\}', '', text)
    # удаляем пробелы внутри текста
    text = re.sub(r'\s+', '', text)
    # удаляем {}
    text = re.sub(r'[{}]', '', text)
    # удаляем слово "Накладная", учитываем оба регистра
    text = re.sub(r'накладная', '', text, flags=re.I)
    # удаляем слово "ТТН", учитываем оба регистра
    text = re.sub(r'ттн', '', text, flags=re.I)

    # возвращаем данные без пробелов в верхнем регистре
    if text == '':
      return None
    if text:
      return text.upper()

  # если не нашлось данных - возвращаем пустое значение
  return None

Функция для поиска номера трекера

In [ ]:
def tracker_number(text):
  # проверка на наличие данных
  if pd.notna(text):
    text = str(text)

    # убираем лишний текст (" от ДД.ММ.ГГГГ")
    text = re.sub(r'\s+[Оо][Тт].*$', '', text)
    # удаляем id
    text = re.sub(r'\{\d{10}\}', '', text)
    # удаляем слово "ТТН", учитываем оба регистра
    text = re.sub(r'ттн', '', text, flags=re.I)
    # удаляем тире
    text = re.sub('-', '', text)
    # удаляем пробелы
    text = re.sub(r'\s+', '', text)
    # удаляем {}
    text = re.sub(r'[{}]', '', text)

    # если текста фактически не осталось - возвращаем пустое значение
    if text == '':
      return None

    # возвращаем очищенный текст в верхнем регистре
    return text.upper()

  # заглушка
  return None

Функция преобразования приложений к договорам

In [ ]:
def app_nums_processing(text):
  # проверка на наличие данных
  if pd.isna(text):
    return text

  # удаляем символ номера
  text = text.replace('№', '').strip()

  # функция ничего не выводит, если аргумент отсутствует
  if not text:
    return text

  # сначала разбиваем на группы по точке с запятой
  groups = [group.strip() for group in text.split(';') if group.strip()]

  result_groups = []

  for group in groups:
    # внутри группы разбиваем по запятым
    numbers = [num.strip() for num in group.split(',') if num.strip()]

    # берем только первую часть до пробела для каждого номера
    processed_numbers = []
    for num in numbers:
      # берем первую часть до пробела, если есть пробел
      first_part = num.split()[0] if ' ' in num else num
      processed_numbers.append(first_part)

    # cобираем группу обратно через запятые
    result_groups.append(', '.join(processed_numbers))

  # cобираем все группы через точку с запятой
  return '; '.join(result_groups)

Функция преобразования договоров

In [ ]:
def contracts_processing(text):
  # проверка на наличие данных
  if pd.isna(text):
    return text

  # удаляем лишнюю часть договора, представляющую собой номер приложения
  text = re.sub(r'_СП\d+', '', text)

  # приводим данные к строковому типу и к верхнему регистру
  text = str(text).upper().strip()

  # функция ничего не выводит, если аргумент отсутствует
  if not text:
    return text

  # удаляем символ номера и точки, заменяем все возможные разделители на ;
  text = text.replace('№', '').replace('.', '').replace('\\', ';').replace(',', ';')

  # удаляем буквенную часть договора
  text = text.replace('ДП_', '').replace('Д_', '').replace('_Р', ' ')

  # заменяем двойные разделители на одинарные
  while ';;' in text:
    text = text.replace(';;', ';')

  # разделяем по точке с запятой
  contracts = [contract.strip() for contract in text.split(';') if contract.strip()]

  # берем только первую часть до пробела для каждого договора
  processed_contracts = []
  for contract in contracts:
    # берем первую часть до пробела, если есть пробел
    first_part = contract.split()[0] if ' ' in contract else contract
    # удаляем слеши, если остались
    first_part = first_part.replace('/', '')
    # проверяем, что строка не пустая
    if first_part:
      processed_contracts.append(first_part)

  # собираем все договоры через точку с запятой
  return '; '.join(processed_contracts)

Функция извлечения кода с учетом приоритетов столбцов

In [ ]:
def extract_code(text):

  # проверка на наличие данных в ячейке столбца - это необходимо для дальнейшего использования в лямбда-функции
  if pd.notna(text):
    text = str(text)

    # поиск кода в тексте - извлекается группа из одной или более цифры подряд из скобок {}
    res = re.search(r'\{(\d+)\}', text)
    if res:
      # в случае нахождения совпадений возвращается первая извлеченная из скобок группа цифр
      return res.group(1)

    # поиск кода в тексте - извлекается группа из ровно 10 цифр подряд без скобок {}
    res = re.search(r'\b(\d{10})\b', text)
    if res:
      # в случае нахождения совпадений возвращается первая извлеченная из скобок группа цифр
      return res.group(1)

  # в случае отсутствия искомого паттерна функция ничего не возвращает
  return None

Функция подсчета СФ

In [ ]:
def analyze_list(lst):

  # обеспечиваем, что lst — список
  if not isinstance(lst, list):
    lst = [lst]

  # по умолчанию ставим метку False на наличие значений и пропусков
  has_nan = False
  has_values = False

  # проверяем наличие и отсутствие пропусков
  for item in lst:
    if item is None or (isinstance(item, float) and np.isnan(item)):
      has_nan = True
    elif item is not None:
      has_values = True

  # найдены и пропуски и значения
  if has_nan and has_values:
    return 'Сч.ф. найдены частично'
  # найдены только пропуски
  elif has_nan and not has_values:
    return 'Не найдено ни одной сч.ф.'
  # найдены только значения
  elif not has_nan:
    return 'Найдены все сч.ф.'

  # заглушка
  return 'Найдены все сч.ф.'

Функция создания ключей с логикой разделения через ";"

In [ ]:
def generate_keys(contracts, applications, another):

  # приводим данные к строчному типу
  contracts = str(contracts)
  applications = str(applications)

  # убираем пробелы
  contracts = contracts.replace(' ', '')#.replace(';', ',')
  applications = applications.replace(' ', '').replace('.', ';')

  # формируем списки
  contracts = contracts.split(';')
  applications = applications.split(';')

  # формируем список с группами приложений (каждая группа - список, т.е. получаем список списков)
  app_list = [[str(num.strip()) for num in lst.split(',')] for lst in applications]

  # это третий аргумент для любого столбца, ячейки которого содержат одно значение
  another_value = str(another).strip()

  # сюда будем складывать найденные ключи
  keys = []

  # случай 1: кол-во договоров = кол-во групп приложений
  if len(contracts) == len(app_list):
    for contract, group_apps in zip(contracts, app_list):
      for app in group_apps:
        key = f'{contract}_{app}_{another_value}'
        keys.append(key)

  # случай 2: 1 договор и >= 1 групп приложений
  elif len(contracts) == 1 and len(app_list) >= 1:
    contract = contracts[0]
    apps = sum(app_list, [])
    for app in apps:
      key = f'{contract}_{app}_{another_value}'
      keys.append(key)

  # случай 3: > 1 договоров и >= 1 групп приложений, при этом кол-во договоров != кол-во групп приложений
  # таких случаев быть не должно, т.к. кол-во договоров должно соответствовать кол-ву групп приложений
  # если такой случай выявлен, то запускается первый сценарий - соответствие будет идти по меньшему кол-ву Д или групп П
  else:
    for contract, group_apps in zip(contracts, app_list):
      for app in group_apps:
        key = f'{contract}_{app}_{another_value}'
        keys.append(key)


  return keys

Функция соединения данных

In [ ]:
def dfs_merge(df_fca_start, df_fca, df_sap):

  # шаг 1 - ищем соответствие по id
  print('\033[1mШаг первый:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_one = df_fca.copy()
  sap_exp_one = df_sap.copy()

  # комбинация ключа Д + П + id, попутно форматирование
  df_step_one.loc[:, 'Ключ'] = df_step_one.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          row['Код']
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_one = df_step_one.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_one['Номер ключа'] = df_step_one.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_one['Фильтр'] = df_step_one['Индекс перевозки'].astype(str) + '_' + df_step_one['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_one['Маркер Y'] = df_step_one['Ключ'].map(df_step_one['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_one = df_step_one[[
      'Индекс перевозки',
      'Код',
      'Дата отгрузки ФАКТ',
      'Стоимость материала, без НДС',
      'Номер приложения',
      'Договор',
      'Номер ТС / накладной',
      'Номер трекера',
      '№ТН',
      'Ключ',
      'Номер ключа',
      'Фильтр',
      'Маркер Y'
      ]]
  df_step_one = df_step_one.drop_duplicates()

  # комбинация ключа Д + П + id, попутно форматирование
  sap_exp_one.loc[:, 'Ключ'] = sap_exp_one.apply(
      lambda row: [
          key for val in [extract_code(row[col]) for col in ['ТС: №ТранспДок', 'ТС: №ТранспСр-ва',
                                                             'ТС: №ЕдОбр']]
          for key in generate_keys(
              contracts_processing(row['ЮрНомер договора на поставку']),
              app_nums_processing(row['№ приложения к договору']),
              val
          )
      ],
      axis=1
  )

  # растаскиваем по строкам
  sap_exp_one = sap_exp_one.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_one = sap_exp_one[['Ключ', 'Номер документа (сч.ф)', 'ЮрНомер договора на поставку', '№ приложения к договору']]
  sap_exp_one = sap_exp_one.drop_duplicates()

  # соединяем по ключам итоговый на первом шаге датафрейм
  df_step_one = df_step_one.merge(sap_exp_one, how='left', on='Ключ')

  # сч.ф., которые найти не получилось, будем искать на дальнейших итерациях
  df_step_one = df_step_one[~df_step_one['Номер документа (сч.ф)'].isna()]

  # удаляем ключи с Nan на конце
  df_step_one = df_step_one[~df_step_one['Ключ'].str.lower().str.endswith('none')]

  # создаем список отработанных значений-фильтров
  # это значит, что такие индексы ключа для каждой строки больше не будут проверяться, т.к. была найдена сч.ф.
  filter = list(df_step_one['Фильтр'])

  print('Длина полученного датафрейма, ключ - id:', len(df_step_one))

  # кол-во задействованных на шаге индексов
  first_step_len = df_step_one['Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len = first_step_len

  # тип соединения
  df_step_one['Тип соединения'] = 'по договору + приложению + id'

  # кол-во найденных на первой итерации уникальных сч.ф.
  num_one = df_step_one['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', first_step_len)
  print('Кол-во найденных сч.ф.:', num_one)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  # шаг 2 - ищем соответствие по договору + приложению + ТТН
  print('\033[1mШаг второй:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_two = df_fca.copy()
  sap_exp_two = df_sap.copy()

  # форматируем датафрейм FCA
  df_step_two.loc[:, 'Ключ'] = df_step_two.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          transport_invoice(row['№ТН'])
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_two = df_step_two.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_two['Номер ключа'] = df_step_two.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_two['Фильтр'] = df_step_two['Индекс перевозки'].astype(str) + '_' + df_step_two['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_two['Маркер Y'] = df_step_two['Ключ'].map(df_step_two['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_two = df_step_two[[
      'Индекс перевозки',
      'Код',
      'Дата отгрузки ФАКТ',
      'Стоимость материала, без НДС',
      'Номер приложения',
      'Договор',
      'Номер ТС / накладной',
      'Номер трекера',
      '№ТН',
      'Ключ',
      'Номер ключа',
      'Фильтр',
      'Маркер Y'
      ]]
  df_step_two = df_step_two.drop_duplicates()

  # форматируем копию датафрейма SAP, созданную для второго шага
  sap_exp_two.loc[:, 'Ключ'] = sap_exp_two.apply(
      lambda row: [
          key for val in [transport_invoice(row[col]) for col in ['ТС: №ТранспДок', 'ТС: №ТранспСр-ва']]
          for key in generate_keys(
              contracts_processing(row['ЮрНомер договора на поставку']),
              app_nums_processing(row['№ приложения к договору']),
              val
          )
      ],
      axis=1
  )

  # растаскиваем по строкам
  sap_exp_two = sap_exp_two.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_two = sap_exp_two[['Ключ', 'Номер документа (сч.ф)', 'ЮрНомер договора на поставку', '№ приложения к договору']]
  sap_exp_two = sap_exp_two.drop_duplicates()

  # соединяем по ключам итоговый на втором шаге датафрейм
  df_step_two = df_step_two.merge(sap_exp_two, how='left', on='Ключ')

  # сч.ф., которые найти не получилось, будем искать на дальнейших итерациях
  df_step_two = df_step_two[~df_step_two['Номер документа (сч.ф)'].isna()]

  # удаляем ключи с Nan на конце
  df_step_two = df_step_two[~df_step_two['Ключ'].str.lower().str.endswith('none')]

  # выводим индексы с найденными ключами для строки из поиска
  df_step_two = df_step_two[~df_step_two['Фильтр'].isin(filter)]

  # дополняем список отработанных значений-фильтров
  filter += list(df_step_two['Фильтр'])

  print('Длина полученного датафрейма, ключ - договор + приложение + ТТН:', len(df_step_two))

  # кол-во задействованных на шаге индексов
  second_step_len = df_step_two['Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len += second_step_len

  # тип соединения
  df_step_two['Тип соединения'] = 'по договору + приложению + ТТН'

  # кол-во найденных на первой итерации уникальных сч.ф.
  num_two = df_step_two['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', second_step_len)
  print('Кол-во найденных сч.ф.:', num_two)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  # шаг 3 - ищем соответствие по договору + приложению + номер трекера
  print('\033[1mШаг третий:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_three = df_fca.copy()
  sap_exp_three = df_sap.copy()

  # форматируем датафрейм FCA
  df_step_three = df_step_three.copy()
  df_step_three.loc[:, 'Ключ'] = df_step_three.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          tracker_number(row['Номер трекера'])
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_three = df_step_three.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_three['Номер ключа'] = df_step_three.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_three['Фильтр'] = df_step_three['Индекс перевозки'].astype(str) + '_' + df_step_three['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_three['Маркер Y'] = df_step_three['Ключ'].map(df_step_three['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_three = df_step_three[[
      'Индекс перевозки',
      'Код',
      'Дата отгрузки ФАКТ',
      'Стоимость материала, без НДС',
      'Номер приложения',
      'Договор',
      'Номер ТС / накладной',
      'Номер трекера',
      '№ТН',
      'Ключ',
      'Номер ключа',
      'Фильтр',
      'Маркер Y'
      ]]
  df_step_three = df_step_three.drop_duplicates()

  # форматируем копию датафрейма SAP, созданную для третьего шага
  sap_exp_three.loc[:, 'Ключ'] = sap_exp_three.apply(
      lambda row: [
          key for val in [tracker_number(row[col]) for col in ['ТС: №ТранспСр-ва', 'ТС: №ТранспДок', 'ТС: №ЕдОбр']]
          for key in generate_keys(
              contracts_processing(row['ЮрНомер договора на поставку']),
              app_nums_processing(row['№ приложения к договору']),
              val
          )
      ],
      axis=1
  )

  # растаскиваем по строкам
  sap_exp_three = sap_exp_three.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_three = sap_exp_three[['Ключ', 'Номер документа (сч.ф)', 'ЮрНомер договора на поставку', '№ приложения к договору']]
  sap_exp_three = sap_exp_three.drop_duplicates()

  # соединяем по ключам итоговый на третьем шаге датафрейм
  df_step_three = df_step_three.merge(sap_exp_three, how='left', on='Ключ')

  # сч.ф., которые найти не получилось, будем искать на дальнейших итерациях
  df_step_three = df_step_three[~df_step_three['Номер документа (сч.ф)'].isna()]

  # # удаляем ключи с Nan на конце
  df_step_three = df_step_three[~df_step_three['Ключ'].str.lower().str.endswith('none')]

  # выводим индексы с найденными ключами для строки из поиска
  df_step_three = df_step_three[~df_step_three['Фильтр'].isin(filter)]

  # дополняем список отработанных значений-фильтров
  filter += list(df_step_three['Фильтр'])

  print('Длина полученного датафрейма, ключ - договор + приложение + номер трекера:', len(df_step_three))

  # кол-во задействованных на шаге индексов
  third_step_len = df_step_three['Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len += third_step_len

  # тип соединения
  df_step_three['Тип соединения'] = 'по договору + приложению + номеру трекера'

  # кол-во найденных на первой итерации уникальных сч.ф.
  num_three = df_step_three['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', third_step_len)
  print('Кол-во найденных сч.ф.:', num_three)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  # шаг 4 - ищем соответствие по договору + приложению + номер ТС
  print('\033[1mШаг четвертый:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_four = df_fca.copy()
  sap_exp_four = df_sap.copy()

  # форматируем датафрейм FCA
  df_step_four = df_step_four.copy()
  df_step_four.loc[:, 'Ключ'] = df_step_four.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          transport_number(row['Номер ТС / накладной'])
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_four = df_step_four.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_four['Номер ключа'] = df_step_four.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_four['Фильтр'] = df_step_four['Индекс перевозки'].astype(str) + '_' + df_step_four['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_four['Маркер Y'] = df_step_four['Ключ'].map(df_step_four['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_four = df_step_four[[
      'Индекс перевозки',
      'Код',
      'Дата отгрузки ФАКТ',
      'Стоимость материала, без НДС',
      'Номер приложения',
      'Договор',
      'Номер ТС / накладной',
      'Номер трекера',
      '№ТН',
      'Ключ',
      'Номер ключа',
      'Фильтр',
      'Маркер Y'
      ]]
  df_step_four = df_step_four.drop_duplicates()

  # форматируем копию датафрейма SAP, созданную для четвертого шага
  sap_exp_four.loc[:, 'Ключ'] = sap_exp_four.apply(
      lambda row: [
          key for val in [transport_number(row[col]) for col in ['ТС: №ТранспСр-ва', 'ТС: №ТранспДок']]
          for key in generate_keys(
              contracts_processing(row['ЮрНомер договора на поставку']),
              app_nums_processing(row['№ приложения к договору']),
              val
          )
      ],
      axis=1
      )

  # растаскиваем по строкам
  sap_exp_four = sap_exp_four.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_four = sap_exp_four[['Ключ', 'Номер документа (сч.ф)', 'ЮрНомер договора на поставку', '№ приложения к договору']]
  sap_exp_four = sap_exp_four.drop_duplicates()

  # соединяем по ключам итоговый на четвертом шаге датафрейм
  df_step_four = df_step_four.merge(sap_exp_four, how='left', on='Ключ')

  # сч.ф., которые найти не получилось, будем искать на дальнейших итерациях
  df_step_four = df_step_four[~df_step_four['Номер документа (сч.ф)'].isna()]

  # удаляем ключи с Nan на конце
  df_step_four = df_step_four[~df_step_four['Ключ'].str.lower().str.endswith('none')]

  # выводим индексы с найденными ключами для строки из поиска
  df_step_four = df_step_four[~df_step_four['Фильтр'].isin(filter)]

  # дополняем список отработанных значений-фильтров
  filter += list(df_step_four['Фильтр'])

  print('Длина полученного датафрейма, ключ - договор + приложение + номер ТС:', len(df_step_four))

  # кол-во задействованных на шаге индексов
  fourth_step_len = df_step_four['Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len += fourth_step_len

  # тип соединения
  df_step_four['Тип соединения'] = 'по договору + приложению + номеру ТС'

  # кол-во найденных на четвертой итерации уникальных сч.ф.
  num_four = df_step_four['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', fourth_step_len)
  print('Кол-во найденных сч.ф.:', num_four)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  # шаг 5 - ищем соответствие по договору + приложению + дата отправки ТС
  print('\033[1mШаг пятый:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_five = df_fca.copy()
  sap_exp_five = df_sap.copy()

  # форматируем датафрейм FCA
  df_step_five = df_step_five.copy()
  df_step_five.loc[:, 'Ключ'] = df_step_five.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          row['Дата отгрузки ФАКТ']
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_five = df_step_five.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_five['Номер ключа'] = df_step_five.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_five['Фильтр'] = df_step_five['Индекс перевозки'].astype(str) + '_' + df_step_five['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_five['Маркер Y'] = df_step_five['Ключ'].map(df_step_five['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_five = df_step_five[[
      'Индекс перевозки',
      'Код',
      'Дата отгрузки ФАКТ',
      'Стоимость материала, без НДС',
      'Номер приложения',
      'Договор',
      'Номер ТС / накладной',
      'Номер трекера',
      '№ТН',
      'Ключ',
      'Номер ключа',
      'Фильтр',
      'Маркер Y'
      ]]
  df_step_five = df_step_five.drop_duplicates()

  # форматируем копию датафрейма SAP, созданную для пятого шага
  sap_exp_five.loc[:, 'Ключ'] = sap_exp_five.apply(
      lambda row: generate_keys(
          contracts_processing(row['ЮрНомер договора на поставку']),
          app_nums_processing(row['№ приложения к договору']),
          row['ТС: Дата отправки']
          ),
      axis=1
      )

  # растаскиваем по строкам
  sap_exp_five = sap_exp_five.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_five = sap_exp_five[['Ключ', 'Номер документа (сч.ф)', 'ЮрНомер договора на поставку', '№ приложения к договору']]
  sap_exp_five = sap_exp_five.drop_duplicates()

  # соединяем по ключам итоговый на пятом шаге датафрейм
  df_step_five = df_step_five.merge(sap_exp_five, how='left', on='Ключ')

  # сч.ф., которые найти не получилось, будем искать на дальнейших итерациях
  df_step_five = df_step_five[~df_step_five['Номер документа (сч.ф)'].isna()]

  # удаляем ключи с Nan на конце
  df_step_five = df_step_five[~df_step_five['Ключ'].str.lower().str.endswith('none')]

  # выводим индексы с найденными ключами для строки из поиска
  df_step_five = df_step_five[~df_step_five['Фильтр'].isin(filter)]

  # дополняем список отработанных значений-фильтров
  filter += list(df_step_five['Фильтр'])

  print('Длина полученного датафрейма, ключ - договор + приложение + дата отправки ТС:', len(df_step_five))

  # кол-во задействованных на шаге индексов
  fifth_step_len = df_step_five.loc[df_step_five['Номер документа (сч.ф)'].notna(), 'Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len += fifth_step_len

  # тип соединения
  df_step_five['Тип соединения'] = 'по договору + приложению + дате отправки ТС'

  # кол-во найденных на пятой итерации уникальных сч.ф.
  num_five = df_step_five['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', fifth_step_len)
  print('Кол-во найденных сч.ф.:', num_five)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  # шаг 6 - ищем соответствие по договору + приложению + стоимости материала
  print('\033[1mШаг шестой:\033[0m')

  # копируем исходные реестры во избежание изменений в них
  df_step_six = df_fca.copy()
  sap_exp_six = df_sap.copy()

  # форматируем датафрейм FCA
  df_step_six = df_step_six.copy()
  df_step_six.loc[:, 'Ключ'] = df_step_six.apply(
      lambda row: generate_keys(
          contracts_processing(row['Договор']),
          app_nums_processing(row['Номер приложения']),
          str(round(row['Стоимость материала, без НДС'] * 1.2, 2))
          ),
      axis=1
      )

  # растаскиваем по строкам
  df_step_six = df_step_six.explode('Ключ').reset_index(drop=True)

  # добавляем номер ключа
  df_step_six['Номер ключа'] = df_step_six.groupby('Индекс перевозки').cumcount() + 1

  # добавляем возможность фильтрации ключей
  df_step_six['Фильтр'] = df_step_six['Индекс перевозки'].astype(str) + '_' + df_step_six['Номер ключа'].astype(str)

  # считаем для каждой строки кол-во раз, которое встречается ключ по ней в таблице
  df_step_six['Маркер Y'] = df_step_six['Ключ'].map(df_step_six['Ключ'].value_counts())

  # удаляем лишние поля и затем дубликаты
  df_step_six = df_step_six[[
      'Индекс перевозки',
      'Код',
      'Дата отгрузки ФАКТ',
      'Стоимость материала, без НДС',
      'Номер приложения',
      'Договор',
      'Номер ТС / накладной',
      'Номер трекера',
      '№ТН',
      'Ключ',
      'Номер ключа',
      'Фильтр',
      'Маркер Y'
      ]]
  df_step_six = df_step_six.drop_duplicates()

  # форматируем копию датафрейма SAP, созданную для шестого шага
  sap_exp_six.loc[:, 'Ключ'] = sap_exp_six.apply(
      lambda row: generate_keys(
          contracts_processing(row['ЮрНомер договора на поставку']),
          app_nums_processing(row['№ приложения к договору']),
          str(round(row['Сумма брутто'], 2))
          ),
      axis=1
      )

  # растаскиваем по строкам
  sap_exp_six = sap_exp_six.explode('Ключ')

  # удаляем лишние поля и затем дубликаты
  sap_exp_six = sap_exp_six[['Ключ', 'Номер документа (сч.ф)', 'ЮрНомер договора на поставку', '№ приложения к договору']]
  sap_exp_six = sap_exp_six.drop_duplicates()

  # соединяем по ключам итоговый на шестом шаге датафрейм
  df_step_six = df_step_six.merge(sap_exp_six, how='left', on='Ключ')

  # удаляем ключи с Nan на конце
  df_step_six = df_step_six[~df_step_six['Ключ'].str.lower().str.endswith('none')]

  # выводим индексы с найденными ключами для строки из поиска
  df_step_six = df_step_six[~df_step_six['Фильтр'].isin(filter)]

  # дополняем список отработанных значений-фильтров
  filter += list(df_step_six['Фильтр'])

  print('Длина полученного датафрейма, ключ - договор + приложение + стоимость материала:', len(df_step_six))

  # кол-во задействованных на шаге индексов
  sixth_step_len = df_step_six.loc[df_step_six['Номер документа (сч.ф)'].notna(), 'Индекс перевозки'].nunique()
  # кол-во задействованных накопительным итогом индексов
  sum_len += sixth_step_len

  # тип соединения
  df_step_six['Тип соединения'] = 'по договору + приложению + стоимости материала'

  # кол-во найденных на шестой итерации уникальных сч.ф.
  num_six = df_step_six['Номер документа (сч.ф)'].nunique()

  print('Кол-во использованных индексов:', sixth_step_len)
  print('Кол-во найденных сч.ф.:', num_six)
  print('Кол-во строк, которые осталось найти:', len(fca)-sum_len)
  print('Кол-во найденных строк накопительным итогом:', sum_len)
  print('% нахождения накопительным итогом:', round(sum_len / len(fca) * 100, 2))
  print('\033[1m----------------------------------------\033[0m')

  print('\033[1mИТОГО:\033[0m')

  # соединяем найденные на предыдущих шагах датафреймы
  res = pd.concat([df_step_one, df_step_two, df_step_three, df_step_four, df_step_five, df_step_six], axis=0, ignore_index=True)

  # удаляем строки с не найденными СФ
  res = res[res['Номер документа (сч.ф)'].notna()]

  # сортируем по индексам строк и индексам ключей в строках для корректного порядке при нахождении сч.ф.
  res = res.sort_values(by=['Индекс перевозки', 'Номер ключа'])

  # оставляем только основные столбцы
  res = res[[
      'Индекс перевозки',
      'Код',
      'Дата отгрузки ФАКТ',
      'Стоимость материала, без НДС',
      '№ приложения к договору',
      'ЮрНомер договора на поставку',
      'Номер ТС / накладной',
      'Номер трекера',
      '№ТН',
      'Маркер Y',
      'Номер документа (сч.ф)'
  ]]

  return res

  # # смотрим по каким ключам сколько соединилось строк
  # print(
  #     res.groupby('Тип соединения')['Номер документа (сч.ф)']
  #      .size()
  #      .reset_index(name='Кол-во строк')
  #      .sort_values(by='Кол-во строк', ascending=False)
  #      .reset_index(drop=True)
  #      )

  # # создаем таблицу с индексами и сч.ф., которую будем цеплять к исходному реестру FCA, попутно меняет float на int
  # res = res.groupby(['Индекс перевозки', 'Маркер Y'], sort=False).agg({
  #   'Номер документа (сч.ф)': lambda x: [int(val) if pd.notna(val) else val for val in x]
  #   }).reset_index()

  # # окончательно схлопываем строки (из наличия маркера Y некоторые строки задублировались)
  # res = res.groupby('Индекс перевозки').agg({
  #   'Маркер Y': 'sum',  # сумма по повтору
  #   'Номер документа (сч.ф)': 'sum',  # объединение списков
  #   }).reset_index()

  # # цепляем к реестру FCA найденные сч.ф.
  # result = df_fca_start.merge(res, how='left', on='Индекс перевозки')

  # print('\033[1m----------------------------------------\033[0m')

  # # общее кол-во найденных сч.ф.
  # invoices = res.explode('Номер документа (сч.ф)')
  # n = invoices['Номер документа (сч.ф)'].nunique()
  # print(f'\033[1mОбщее кол-во найденных сч.ф.:\033[0m {n}')
  # print(f'\033[1m% найденных строк\033[0m {round(sum_len / len(fca) * 100, 2)}')

  # return result

# Загрузка и предобработка данных

Загрузка данных

In [ ]:
# загрузка данных
fca = pd.read_excel('Ноябрь 2025.xlsx', usecols=[
    'Код',
    'Дата отгрузки ФАКТ',
    'Номер приложения',
    'Договор',
    'Номер ТС / накладной',
    'Номер трекера',
    '№ТН',
    'Стоимость материала, без НДС',
    ])

sap_519 = pd.read_excel('519 мес.XLSX', usecols = [
    'ЮрНомер договора на поставку',
    '№ приложения к договору',
    'Номер документа (сч.ф)',
    'ТС: №ТранспСр-ва',
    'ТС: №ТранспДок',
    'ТС: Дата отправки',
    'ТС: №ЕдОбр',
    'Сумма брутто'
    ], dtype={'№ приложения к договору': str})

print('Длина FCA:', len(fca))
print('Длина SAP 519:', len(sap_519))

Длина FCA: 1690
Длина SAP 519: 3671


In [ ]:
# добавляем индекс каждой строке реестра FCA
fca.insert(0, 'Индекс перевозки', np.arange(0, len(fca)))

In [ ]:
# поля, которые блок FCA должен заполнить данными
cols_revision = [
    'Код',
    'Договор',
    'Номер приложения',
    'Дата отгрузки ФАКТ',
    ]

# таблица, которая будет направлена блоку FCA на дозаполнение
fca_revision = fca[fca[cols_revision].isnull().any(axis=1)]

Преобразование реестра FCA

In [ ]:
# избавляемся от лишних скобок в поле id, ограничиваем длину id 10 символами
fca['Код'] = fca['Код'].str.replace(r'[{}]', '', regex=True).str[:10]

# приводим данные по стоимостям в текстовый тип данных (техническое решение, чтобы привести цифры в единый вид)
fca['Стоимость материала, без НДС'] = fca['Стоимость материала, без НДС'].astype(str)

# избавляемся от лишних пробелов и приводим цифры в единый вид
fca['Стоимость материала, без НДС'] = fca['Стоимость материала, без НДС'].str.replace(',', '.').str.replace(r'\s+', '', regex=True).str.replace(r'[^\d.]+', '', regex=True)

# меняем тип данных на числовой
fca['Стоимость материала, без НДС'] = pd.to_numeric(fca['Стоимость материала, без НДС'], errors='coerce')

# подстраховка
fca['Стоимость материала, без НДС'] = fca['Стоимость материала, без НДС'].astype(float)

Преобразование выгрузки SAP 519

In [ ]:
# избавляемся от всех строк SAP 519, где нет СФ
sap_519 = sap_519[sap_519['Номер документа (сч.ф)'].notna()]


# меняем тип данных на текстовый
# это дополнительная мера, т.к. тип данных в любом случае меняется внутри функций
cols_fca_str = ['Номер ТС / накладной', 'Номер приложения', 'Номер трекера', '№ТН']
for col in cols_fca_str:
  fca[col] = fca[col].where(fca[col].isna(), fca[col].astype('str'))

cols_sap_str = [
    'ТС: №ТранспСр-ва',
    'ТС: №ТранспДок',
    'ТС: №ЕдОбр',
    'ЮрНомер договора на поставку',
    '№ приложения к договору',
    ]
for col in cols_sap_str:
  sap_519[col] = sap_519[col].where(sap_519[col].isna(), sap_519[col].astype('str'))

# на всякий случай присваиваем СФ тип данных int
sap_519['Номер документа (сч.ф)'] = sap_519['Номер документа (сч.ф)'].astype(int)

# преобразуем поля из текствого формата в списки
col_list = ['ТС: №ТранспСр-ва', 'ТС: №ТранспДок', 'ТС: №ЕдОбр']
sap_519[col_list] = sap_519[col_list].applymap(
    lambda x: [item.strip() for item in x.split(',')] if pd.notna(x) else []
)

# растаскиваем по строкам
sap_519_exp = sap_519.explode('ТС: №ТранспСр-ва').explode('ТС: №ТранспДок').explode('ТС: №ЕдОбр')

# приводим данные по стоимостям в текстовый тип данных (техническое решение, чтобы привести цифры в единый вид)
sap_519_exp['Сумма брутто'] = sap_519_exp['Сумма брутто'].astype(str)

# избавляемся от лишних пробелов и приводим цифры в единый вид
sap_519_exp['Сумма брутто'] = sap_519_exp['Сумма брутто'].str.replace(',', '.').str.replace(r'\s+', '', regex=True).str.replace(r'[^\d.]+', '', regex=True)

# меняем тип данных на числовой
sap_519_exp['Сумма брутто'] = pd.to_numeric(sap_519_exp['Сумма брутто'], errors='coerce')

# подстраховка
sap_519_exp['Сумма брутто'] = sap_519_exp['Сумма брутто'].astype(float)

# чистим столбцы
for col in col_list:
    mask = sap_519_exp[col].str.lower() == 'нет данных'
    sap_519_exp.loc[mask, col] = None

/tmp/ipython-input-1086859690.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sap_519[col_list] = sap_519[col_list].applymap(


In [ ]:
# исключаем часть реестра FCA, отправленную на доработку
fca_clean = fca[~fca['Индекс перевозки'].isin(fca_revision['Индекс перевозки'])]

In [ ]:
# соединяем реестры
df_res = dfs_merge(fca, fca_clean, sap_519_exp)

Шаг первый:
Длина полученного датафрейма, ключ - id: 617
Кол-во использованных индексов: 504
Кол-во найденных сч.ф.: 615
Кол-во строк, которые осталось найти: 1186
Кол-во найденных строк накопительным итогом: 504
% нахождения накопительным итогом: 29.82
----------------------------------------
Шаг второй:
Длина полученного датафрейма, ключ - договор + приложение + ТТН: 16
Кол-во использованных индексов: 9
Кол-во найденных сч.ф.: 16
Кол-во строк, которые осталось найти: 1177
Кол-во найденных строк накопительным итогом: 513
% нахождения накопительным итогом: 30.36
----------------------------------------
Шаг третий:
Длина полученного датафрейма, ключ - договор + приложение + номер трекера: 32
Кол-во использованных индексов: 23
Кол-во найденных сч.ф.: 32
Кол-во строк, которые осталось найти: 1154
Кол-во найденных строк накопительным итогом: 536
% нахождения накопительным итогом: 31.72
----------------------------------------
Шаг четвертый:
Длина полученного датафрейма, ключ - договор + пр

In [ ]:
# переименовываем столбцы
df_res = df_res.rename(columns={'ЮрНомер договора на поставку':'Договор', '№ приложения к договору':'Номер приложения'})

# Преобразование полученных данных

In [ ]:
# маркер Y

# это метка для случая, когда M одинаковых ключей и N сч.ф.
# т.е. проблема, что нельзя установить точное соответствие между сч.ф. и ключом
# в идеале такое должно быть только на итерациях 4 и 5 - по номеру ТС и дате отправки
df_res['Маркер Y'] = np.where((df_res['Маркер Y'] == 1) | (df_res['Маркер Y'].isna()), 'N', 'Y')

In [ ]:
# маркер X

# это проверка того, найдены ли все сч.ф. или нет
df_res['Маркер X'] = df_res['Номер документа (сч.ф)'].apply(analyze_list)

# корректируем маркер Y - не будем помечать им строки без сч.ф.
df_res.loc[df_res['Маркер X'] == 'Не найдено ни одной сч.ф.', 'Маркер Y'] = 'N'

In [ ]:
# функция возвращения фигурных скобок
def curly_brace(id):
  return '{' + str(id) + '}'

# возвращаем фигурные скобки
df_res['Код'] = df_res['Код'].apply(curly_brace)

In [ ]:
# добавляем дополнительный уникальный идентификатор
df_res.insert(loc=2, column='Дополнительный id', value=(df_res['Код'] + '_' + df_res['Договор'] + '_' + df_res['Номер приложения']))

In [ ]:
# удаляем промежуточный столбец "Маркер Х"
df_res = df_res.drop('Маркер X', axis=1)

**Сохранение результатов**

In [ ]:
datasets = [df_res]
sheet_names = ['FCA']
excel_filename = 'Модель данных.xlsx'

with pd.ExcelWriter(excel_filename) as writer:
  for data, sheet in zip(datasets, sheet_names):
    df = pd.DataFrame(data)
    df.to_excel(writer, sheet_name=sheet, header=True, index=False)